# 학생 이동 및 교환 비용 행렬 계산

In [1]:
import pandas as pd
df = pd.read_excel("최종_그룹_배정_결과.xlsx", sheet_name="신규반번호순")
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_우울_T점수,정서적 민감성_우울_수준,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,45,보통,48,보통,32,낮음,62,높음,1,1
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,2
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,3
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,33,낮음,36,낮음,32,낮음,36,낮음,1,4
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,33,낮음,36,낮음,32,낮음,36,낮음,1,5


In [2]:
# 초기 그룹 컬럼 추가하기 (랜덤 숫자 기반)
import numpy as np
df['초기그룹'] = np.random.randint(0, 11, size=len(df))
df['상담필요'] = np.random.randint(0, 2, size=len(df))
df.head()

,학교명,학년반번호,성별,이름,기초학습역량_언어_T점수,기초학습역량_언어_수준,기초학습역량_논리수학_T점수,기초학습역량_논리수학_수준,기초학습역량_공간_T점수,기초학습역량_공간_수준,...,정서적 민감성_부정적 \n대인정서_T점수,정서적 민감성_부정적 \n대인정서_수준,정서적 민감성_예민성_T점수,정서적 민감성_예민성_수준,정서적 민감성_과잉행동_T점수,정서적 민감성_과잉행동_수준,신규반,번호,초기그룹,상담필요
0,장대중학교,10516,1,권원준,23,부족,23,부족,45,낮음,...,48,보통,32,낮음,62,높음,1,1,5,0
1,장대중학교,10505,2,김서윤,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,2,0,0
2,장대중학교,10203,2,김소희,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,3,8,1
3,장대중학교,10605,2,김예은,81,최우수,86,최우수,76,최우수,...,36,낮음,32,낮음,36,낮음,1,4,8,1
4,장대중학교,10615,1,김완,32,매우 낮음,31,매우 낮음,26,부족,...,36,낮음,32,낮음,36,낮음,1,5,2,0


In [3]:
def array_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    이산형 변수의 ideal 대비 actual 차이(diff)와 방향(sign)을 계산하여
    3차원 numpy 배열로 반환하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    selected_discrete_variable : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    diff_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      diff_matrix[g, v, c] = 그룹 g, 변수 v, 카테고리 c 의 diff 값
      diff = ideal_count - actual_count

    sign_matrix : np.ndarray, shape (n_groups, n_vars, n_categories)
      sign_matrix[g, v, c] = diff의 부호 (+1, 0, -1)

    Notes
    -----
    해당 함수는 기존 딕셔너리 구조를 행렬 구조로 바꿔 compute_discrete_cost 계산을 벡터화하기 위해 수정함
    모든 변수중 가장 많은 카테고리 수에 맞춰 0으로 패딩하여 shape이 동일하도록 구성함
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np
    # 확인해보니 반환값이 그룹 번호 순으로 안나와서 정렬 추가
    groups = sorted(list(group_freq.keys()))
    n_groups = len(groups)
    vars_list = selected_discrete_variable
    n_vars = len(vars_list)
    var_to_idx = {var: idx for idx, var in enumerate(vars_list)} # 예시 {성별: 0, 상담필요: 1}
    category_to_idx = {}
    max_categories = 0
    for var in vars_list:
        categories = set(ideal_freq['population'][var].keys()) # 예시 var 성별이면 {1,2}, 상담필요면 {0,1}
        categories = sorted(list(categories))
        category_to_idx[var] = {cat: idx for idx, cat in enumerate(categories)} # 예시 {성별: {1:0, 2:1}, 상담필요: {0:0, 1:1}}
        if len(categories) > max_categories:
            max_categories = len(categories) # 가장 많은 카테고리 수 갱신
    # 행렬 초기화
    diff_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    sign_matrix = np.zeros((n_groups, n_vars, max_categories), dtype=float) # 여기서 padding 발생
    # 행렬 채우기
    try:
        for g_idx, g in enumerate(groups):
            for var in vars_list:
                v_idx = var_to_idx[var]
                cats = category_to_idx[var]
                for cat, c_idx in cats.items():
                    ideal_count = ideal_freq['population'][var].get(cat, 0)
                    acutual_count = group_freq[g][var].get(cat, 0)
                    diff = ideal_count - acutual_count
                    diff_matrix[g_idx, v_idx, c_idx] = diff
                    sign_matrix[g_idx, v_idx, c_idx] = np.sign(diff)
    except Exception as e:
        print("Error during array_diff_and_sign execution:", str(e))
        raise e
    return diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups

In [4]:
# ideal_freq, group_freq, selected_discrete_variable
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
ideal_freq = compute_ideal_discrete_freq(df, ['성별', '상담필요'])
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
print("Ideal Frequency:", ideal_freq)
print("Group Frequency:", group_freq)

Ideal Frequency: {'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
Group Frequency: {np.int32(5): {'성별': {1: 14, 2: 12}, '상담필요': {1: 16, 0: 10}}, np.int32(0): {'성별': {2: 12, 1: 4}, '상담필요': {0: 9, 1: 7}}, np.int32(8): {'성별': {2: 14, 1: 12}, '상담필요': {1: 18, 0: 8}}, np.int32(2): {'성별': {1: 11, 2: 6}, '상담필요': {0: 9, 1: 8}}, np.int32(9): {'성별': {1: 25, 2: 15}, '상담필요': {0: 25, 1: 15}}, np.int32(4): {'성별': {1: 10, 2: 10}, '상담필요': {0: 11, 1: 9}}, np.int32(1): {'성별': {2: 19, 1: 14}, '상담필요': {1: 19, 0: 14}}, np.int32(6): {'성별': {1: 15, 2: 14}, '상담필요': {0: 19, 1: 10}}, np.int32(10): {'성별': {2: 13, 1: 13}, '상담필요': {0: 14, 1: 12}}, np.int32(7): {'성별': {2: 21, 1: 11}, '상담필요': {1: 17, 0: 15}}, np.int32(3): {'성별': {1: 18, 2: 10}, '상담필요': {0: 14, 1: 14}}}


In [5]:
diff_matrix, sign_matrix, var_to_idx, category_to_idx, groups = array_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
print(diff_matrix) # 그룹별 변수별 카테고리별 diff 값
print(sign_matrix) # 그룹별 변수별 카테고리별 sign 값
print(var_to_idx) # 변수명과 인덱스 매핑
print(category_to_idx) # 변수별 카테고리명과 인덱스 매핑
print(groups) # 그룹 번호 리스트

[[[  9.36363636   1.27272727]
  [  4.45454545   6.18181818]]

 [[ -0.63636364  -5.72727273]
  [ -0.54545455  -5.81818182]]

 [[  2.36363636   7.27272727]
  [  4.45454545   5.18181818]]

 [[ -4.63636364   3.27272727]
  [ -0.54545455  -0.81818182]]

 [[  3.36363636   3.27272727]
  [  2.45454545   4.18181818]]

 [[ -0.63636364   1.27272727]
  [  3.45454545  -2.81818182]]

 [[ -1.63636364  -0.72727273]
  [ -5.54545455   3.18181818]]

 [[  2.36363636  -7.72727273]
  [ -1.54545455  -3.81818182]]

 [[  1.36363636  -0.72727273]
  [  5.45454545  -4.81818182]]

 [[-11.63636364  -1.72727273]
  [-11.54545455  -1.81818182]]

 [[  0.36363636   0.27272727]
  [ -0.54545455   1.18181818]]]
[[[ 1.  1.]
  [ 1.  1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1.  1.]
  [-1. -1.]]

 [[ 1.  1.]
  [ 1.  1.]]

 [[-1.  1.]
  [ 1. -1.]]

 [[-1. -1.]
  [-1.  1.]]

 [[ 1. -1.]
  [-1. -1.]]

 [[ 1. -1.]
  [ 1. -1.]]

 [[-1. -1.]
  [-1. -1.]]

 [[ 1.  1.]
  [-1.  1.]]]
{'성별': 0, '상담필요': 1}
{'성별': {1: 

In [6]:
# 기존 딕셔너리로 계산하는 함수 가지고 와서 array_diff_and_sign와 동일한지 확인
import pprint
from cost_group_move_v3 import compute_ideal_discrete_freq, compute_group_discrete_freq
group_freq = compute_group_discrete_freq(df, ['성별', '상담필요'])
pprint.pprint(group_freq)
print(ideal_freq)
def compute_group_diff_and_sign(ideal_freq, group_freq, selected_discrete_variable):
    """
    각 그룹별 이산형 변수의 차이(diff)와 방향(sign)을 계산하는 함수

    Parameters
    ----------
    ideal_freq : dict
        이상적인 빈도 정보 (compute_ideal_discrete_freq()의 결과)
        예시:
        {
            'population': {
                '성별_명렬표': {1: 17.5, 2: 17.5},
                '상담 필요': {0: 30.0, 1: 5.0}
            }
        }

    group_freq : dict
        실제 그룹별 범주 빈도 정보 (compute_group_discrete_freq()의 결과)
        예시:
        {
            0: {'성별_명렬표': {1: 10, 2: 8}, '상담 필요': {0: 15, 1: 3}},
            1: {'성별_명렬표': {1: 12, 2: 6}, '상담 필요': {0: 14, 1: 4}}
        }

    discrete_vars : list
        계산 대상 이산형 변수명 리스트
        예: ['성별_명렬표', '상담 필요']

    Returns
    -------
    group_diff_cost : dict
        각 그룹의 변수별 diff와 sign을 담은 딕셔너리
        구조 예시:
        {
            0: {
                '성별_명렬표_diff': {1: 7.5, 2: 9.5},
                '성별_명렬표_sign': {1: +1.0, 2: +1.0},
                '상담 필요_diff': {0: 15.0, 1: 2.0},
                '상담 필요_sign': {0: +1.0, 1: +1.0}
            },
            1: { ... },
            ...
        }

    Notes
    -----
    - diff = ideal_count - actual_count
      → 양수(+)면 ideal보다 부족, 음수(–)면 ideal보다 과잉.
    - sign = np.sign(diff)
      → +1 = 증가 필요, –1 = 감소 필요, 0 = 이상적.
    - swap 후보 탐색 시,
      sign이 반대인 그룹끼리 교환 가능성 탐색에 활용함.
    """
    import numpy as np

    group_diff_cost = {}
    try:
        for g in group_freq.keys():
            group_diff_cost[g] = {}

            for var in selected_discrete_variable:
                # diff, sign 각각을 별도 dict로 저장
                diff_dict = {}
                sign_dict = {}

                # 각 범주(key)별로 ideal 대비 actual 차이 계산
                for key, ideal_count in ideal_freq['population'][var].items():
                    actual_count = group_freq[g][var].get(key, 0)
                    diff = ideal_count - actual_count
                    diff_dict[key] = diff
                    sign_dict[key] = np.sign(diff) # 이상치보다 작으면 +1, 크면 -1, 같으면 0

                # 변수별 결과를 group_diff_cost에 저장
                group_diff_cost[g][f'{var}_diff'] = diff_dict
                group_diff_cost[g][f'{var}_sign'] = sign_dict
    except Exception as e:
        print("Error during compute_group_diff_and_sign execution:", str(e))
        raise e
    return group_diff_cost
group_diff_cost = compute_group_diff_and_sign(ideal_freq, group_freq, ['성별', '상담필요'])
import pprint
pprint.pprint(group_diff_cost)
# 출력값 확인해보니 diff_matrix와 group_diff_cost와 값은 동일하나 그룹 순서가 안맞아서 array_diff_and_sign에 그룹 정렬 추가함
# 행렬의 변수 순서가 다르긴 하지만 이건 var_to_idx로 매핑 가능함

{np.int32(0): {'상담필요': {0: 9, 1: 7}, '성별': {1: 4, 2: 12}},
 np.int32(1): {'상담필요': {0: 14, 1: 19}, '성별': {1: 14, 2: 19}},
 np.int32(2): {'상담필요': {0: 9, 1: 8}, '성별': {1: 11, 2: 6}},
 np.int32(3): {'상담필요': {0: 14, 1: 14}, '성별': {1: 18, 2: 10}},
 np.int32(4): {'상담필요': {0: 11, 1: 9}, '성별': {1: 10, 2: 10}},
 np.int32(5): {'상담필요': {0: 10, 1: 16}, '성별': {1: 14, 2: 12}},
 np.int32(6): {'상담필요': {0: 19, 1: 10}, '성별': {1: 15, 2: 14}},
 np.int32(7): {'상담필요': {0: 15, 1: 17}, '성별': {1: 11, 2: 21}},
 np.int32(8): {'상담필요': {0: 8, 1: 18}, '성별': {1: 12, 2: 14}},
 np.int32(9): {'상담필요': {0: 25, 1: 15}, '성별': {1: 25, 2: 15}},
 np.int32(10): {'상담필요': {0: 14, 1: 12}, '성별': {1: 13, 2: 13}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
{np.int32(0): {'상담필요_diff': {0: 4.454545454545455, 1: 6.181818181818182},
               '상담필요_sign': {0: np.float64(1.0), 1: np.float64(1.0)},
               '성별_diff': {1: 9.363636363636363, 2: 1.

In [7]:
# 출발 그룹 설정이 제대로 되는지 확인
from cost_group_move_v3 import compute_group_total_cost
pprint.pprint(group_freq)
print(ideal_freq)
group_total_cost, _ = compute_group_total_cost(ideal_freq, group_freq, ['성별', '상담필요'])
pprint.pprint(group_total_cost)
source_group_idx = min(group_total_cost, key=group_total_cost.get)
print("Source Group Index with Minimum Total Cost:", source_group_idx)

{np.int32(0): {'상담필요': {0: 9, 1: 7}, '성별': {1: 4, 2: 12}},
 np.int32(1): {'상담필요': {0: 14, 1: 19}, '성별': {1: 14, 2: 19}},
 np.int32(2): {'상담필요': {0: 9, 1: 8}, '성별': {1: 11, 2: 6}},
 np.int32(3): {'상담필요': {0: 14, 1: 14}, '성별': {1: 18, 2: 10}},
 np.int32(4): {'상담필요': {0: 11, 1: 9}, '성별': {1: 10, 2: 10}},
 np.int32(5): {'상담필요': {0: 10, 1: 16}, '성별': {1: 14, 2: 12}},
 np.int32(6): {'상담필요': {0: 19, 1: 10}, '성별': {1: 15, 2: 14}},
 np.int32(7): {'상담필요': {0: 15, 1: 17}, '성별': {1: 11, 2: 21}},
 np.int32(8): {'상담필요': {0: 8, 1: 18}, '성별': {1: 12, 2: 14}},
 np.int32(9): {'상담필요': {0: 25, 1: 15}, '성별': {1: 25, 2: 15}},
 np.int32(10): {'상담필요': {0: 14, 1: 12}, '성별': {1: 13, 2: 13}}}
{'population': {'성별': {1: 13.363636363636363, 2: 13.272727272727273}, '상담필요': {0: 13.454545454545455, 1: 13.181818181818182}}}
{np.int32(0): np.float64(147.35537190082644),
 np.int32(1): np.float64(-67.35537190082644),
 np.int32(2): np.float64(105.17355371900827),
 np.int32(3): np.float64(-11.75206611570248),
 np.int32(4): 

In [8]:
# 출발 학생 목록 가지고오기
source_group_df = df[df['초기그룹']==source_group_idx]
print(source_group_df['상담필요'].value_counts())
print(source_group_df['성별'].value_counts())

상담필요
0    25
1    15
Name: count, dtype: int64
성별
1    25
2    15
Name: count, dtype: int64


In [9]:
# 타깃그룹 설정
groups = df['초기그룹'].unique()
target_group_list = [g for g in groups if g != source_group_idx]
print("Target Group List (excluding source group):", target_group_list)

Target Group List (excluding source group): [np.int32(5), np.int32(0), np.int32(8), np.int32(2), np.int32(4), np.int32(1), np.int32(6), np.int32(10), np.int32(7), np.int32(3)]


In [10]:
var_to_idx

{'성별': 0, '상담필요': 1}

In [11]:
category_to_idx

{'성별': {1: 0, 2: 1}, '상담필요': {0: 0, 1: 1}}

In [12]:
diff_matrix
# 해당 배열은 아래와 같음
# G, V, C 순서로 되어있음
# G : 그룹 번호
# V : 변수 번호 (0: 성별, 1: 상담필요)
# C : 카테고리 번호 (성별: 0-남,1-여 / 상담필요: 0-필요없음,1-필요함)

array([[[  9.36363636,   1.27272727],
        [  4.45454545,   6.18181818]],

       [[ -0.63636364,  -5.72727273],
        [ -0.54545455,  -5.81818182]],

       [[  2.36363636,   7.27272727],
        [  4.45454545,   5.18181818]],

       [[ -4.63636364,   3.27272727],
        [ -0.54545455,  -0.81818182]],

       [[  3.36363636,   3.27272727],
        [  2.45454545,   4.18181818]],

       [[ -0.63636364,   1.27272727],
        [  3.45454545,  -2.81818182]],

       [[ -1.63636364,  -0.72727273],
        [ -5.54545455,   3.18181818]],

       [[  2.36363636,  -7.72727273],
        [ -1.54545455,  -3.81818182]],

       [[  1.36363636,  -0.72727273],
        [  5.45454545,  -4.81818182]],

       [[-11.63636364,  -1.72727273],
        [-11.54545455,  -1.81818182]],

       [[  0.36363636,   0.27272727],
        [ -0.54545455,   1.18181818]]])

In [13]:
# 배열의 차원 예시는 아래와 같다
# G : 그룹 번호
diff_matrix[source_group_idx] # 출발 그룹의 변수별 카테고리별 diff 값

array([[-11.63636364,  -1.72727273],
       [-11.54545455,  -1.81818182]])

In [14]:
# V : 변수 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값
s_row = source_group_df.iloc[0]
print(s_row[['성별', '상담필요']])
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    print(f"Variable: {var}, Index: {v_idx}")
    print(diff_matrix[source_group_idx, v_idx]) # 출발 그룹의 첫번째 변수(성별)의 카테고리별 diff 값

성별      1
상담필요    0
Name: 5, dtype: object
Variable: 성별, Index: 0
[-11.63636364  -1.72727273]
Variable: 상담필요, Index: 1
[-11.54545455  -1.81818182]


In [15]:
# C : 카테고리 번호
# 출발 그룹 중 첫번째 학생이 속한 변수(성별)의 카테고리별 diff 값 중
# 해당 학생이 속한 카테고리의 diff 값
for var in ['성별', '상담필요']:
    v_idx = var_to_idx[var]
    cat_value = s_row[var]
    c_idx = category_to_idx[var][cat_value]
    print(f"Variable: {var}, Category Value: {cat_value}, Category Index: {c_idx}")
    print(diff_matrix[source_group_idx, v_idx, c_idx]) # 출발 그룹의 첫번째 변수(성별)의 해당 카테고리의 diff 값

Variable: 성별, Category Value: 1, Category Index: 0
-11.636363636363637
Variable: 상담필요, Category Value: 0, Category Index: 0
-11.545454545454545


In [16]:
# 이동 전 모든 그룹의 diff_matrix 확인
for group_idx in range(diff_matrix.shape[0]):
    print(f"Group Index: {group_idx}")
    print(diff_matrix[group_idx])

Group Index: 0
[[9.36363636 1.27272727]
 [4.45454545 6.18181818]]
Group Index: 1
[[-0.63636364 -5.72727273]
 [-0.54545455 -5.81818182]]
Group Index: 2
[[2.36363636 7.27272727]
 [4.45454545 5.18181818]]
Group Index: 3
[[-4.63636364  3.27272727]
 [-0.54545455 -0.81818182]]
Group Index: 4
[[3.36363636 3.27272727]
 [2.45454545 4.18181818]]
Group Index: 5
[[-0.63636364  1.27272727]
 [ 3.45454545 -2.81818182]]
Group Index: 6
[[-1.63636364 -0.72727273]
 [-5.54545455  3.18181818]]
Group Index: 7
[[ 2.36363636 -7.72727273]
 [-1.54545455 -3.81818182]]
Group Index: 8
[[ 1.36363636 -0.72727273]
 [ 5.45454545 -4.81818182]]
Group Index: 9
[[-11.63636364  -1.72727273]
 [-11.54545455  -1.81818182]]
Group Index: 10
[[ 0.36363636  0.27272727]
 [-0.54545455  1.18181818]]


In [17]:
# 출발 그룹에서 모든 학생들이 도착 그룹으로 이동했을때 diff_matrix 계산
'''
어떻게 생각해야할까?
출발 그룹 중 하나의 학생이
도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?
그면 차원이 어떻게 될까?
    출발 그룹 학생 수 만큼 차원이 늘어나고
    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?
    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?
    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데
'''
# 우선 차원 틀 먼저 만들어보자
# N : 출발 그룹 학생 수
# G : 도착 그룹 수 (출발 그룹 제외해야하지만 계산을 위해 모든 그룹 포함)
# V : 변수 수
# C : 가장 많은 카테고리 수

'\n어떻게 생각해야할까?\n출발 그룹 중 하나의 학생이\n도착 그룹에 이동했을 때의 diff_matrix 계산해야하나?\n그면 차원이 어떻게 될까?\n    출발 그룹 학생 수 만큼 차원이 늘어나고\n    그 학생이 도착 그룹으로 이동해야하니까 도착 그룹 수만큼 늘어나야하나?\n    아니면 출발 그룹을 제외하고 도착 그룹 수만큼 늘어나야하는데 나중에 계산하려면 모든 그룹을 고려해야하나?\n    이동한 후 해당 학생의 변수와 카테고리를 고려해 도착 그릅의 diff_matrix 계산해야하는데\n'

In [130]:
# 이동 후 diff_matrix 배열 반영
N = source_group_df.shape[0]
G = len(target_group_list) # 타깃 그룹만 -> 출발 그룹은 따로 배열 계산하는것으로 진행 (두개를 같은 배열에 두니까 복잡함...)
V = diff_matrix.shape[1]
C = diff_matrix.shape[2]
print(f"N (출발 그룹 학생 수): {N}, G (전체 그룹 수): {G}, V (변수 수): {V}, C (최대 카테고리 수): {C}")

N (출발 그룹 학생 수): 40, G (전체 그룹 수): 10, V (변수 수): 2, C (최대 카테고리 수): 2


In [131]:
# 출발 그룹 df 인덱스와 행렬 인덱스 매핑
studentidx_to_arrayidx = {}
for idx, student_idx in enumerate(source_group_df.index):
    studentidx_to_arrayidx[student_idx] = idx
print("Source Group DataFrame Index to Array Index Mapping:", studentidx_to_arrayidx)
# 출발 그룹 df 인덱스와 행렬 인덱스 역매핑
arrayidx_to_studentidx = {}
for student_idx, array_idx in studentidx_to_arrayidx.items():
    arrayidx_to_studentidx[array_idx] = student_idx
print("Array Index to Source Group DataFrame Index Mapping:", arrayidx_to_studentidx)

Source Group DataFrame Index to Array Index Mapping: {5: 0, 9: 1, 21: 2, 27: 3, 29: 4, 39: 5, 57: 6, 59: 7, 76: 8, 81: 9, 87: 10, 88: 11, 89: 12, 99: 13, 110: 14, 111: 15, 117: 16, 123: 17, 125: 18, 128: 19, 129: 20, 137: 21, 143: 22, 150: 23, 154: 24, 172: 25, 183: 26, 185: 27, 208: 28, 216: 29, 218: 30, 221: 31, 224: 32, 225: 33, 230: 34, 237: 35, 240: 36, 241: 37, 260: 38, 280: 39}
Array Index to Source Group DataFrame Index Mapping: {0: 5, 1: 9, 2: 21, 3: 27, 4: 29, 5: 39, 6: 57, 7: 59, 8: 76, 9: 81, 10: 87, 11: 88, 12: 89, 13: 99, 14: 110, 15: 111, 16: 117, 17: 123, 18: 125, 19: 128, 20: 129, 21: 137, 22: 143, 23: 150, 24: 154, 25: 172, 26: 183, 27: 185, 28: 208, 29: 216, 30: 218, 31: 221, 32: 224, 33: 225, 34: 230, 35: 237, 36: 240, 37: 241, 38: 260, 39: 280}


In [132]:
# G가 결국 어떠한 값이 정확한 도착 그룹 번호인지 명확하지 않아서
# 맵칭 데이터가 필요함
targetgroup_to_arrayidx = {}
for idx, g in enumerate(target_group_list):
    targetgroup_to_arrayidx[g] = idx
print(targetgroup_to_arrayidx)
# key값과 value값이 바뀐것도 필요한가?
arrayidx_to_targetgroup = {}
for arrayidx, target_group in zip(range(G), target_group_list):
    arrayidx_to_targetgroup[arrayidx] = target_group
print(arrayidx_to_targetgroup)

{np.int32(0): 0, np.int32(1): 1, np.int32(2): 2, np.int32(3): 3, np.int32(4): 4, np.int32(5): 5, np.int32(6): 6, np.int32(7): 7, np.int32(8): 8, np.int32(10): 9}
{0: np.int32(0), 1: np.int32(1), 2: np.int32(2), 3: np.int32(3), 4: np.int32(4), 5: np.int32(5), 6: np.int32(6), 7: np.int32(7), 8: np.int32(8), 9: np.int32(10)}


In [133]:
# 확인용
print(sorted(target_group_list))
print(s_row['초기그룹'])

[np.int32(0), np.int32(1), np.int32(2), np.int32(3), np.int32(4), np.int32(5), np.int32(6), np.int32(7), np.int32(8), np.int32(10)]
9


In [134]:
# 출발그룹 학생들이 도착그룹으로 이동했을 때의 diff 변화량 계산
after_target_diff_delta_matrix = np.zeros((N, G, V, C), dtype=float)  # N x G x V x C 형태의 배열 초기화

for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(G): # 도착 그룹 인덱스
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_target_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = -1.0
# 확인용
print(after_target_diff_delta_matrix.shape)
print(len(source_group_df))

(40, 10, 2, 2)
40


In [136]:
after_target_diff_delta_matrix[:, 0, :, :] # 각각의 학생이 첫번째 도착 그룹으로 이동했을 때의 diff 변화량

array([[[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., -1.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[ 0., -1.],
        [-1.,  0.]],

       [[-1.,  0.],
        [-1.,  0.]],

       [[ 0., -1.],
        [ 0., -1.]],

       [[-1.,  0.],
        [ 0., 

In [144]:
# 각각의 학생이 출발 그룹에서 도착그룹으로 이동했을 때 출발 그룹의 diff 변화량
after_source_diff_delta_matrix = np.zeros((N, 1, V, C), dtype=float)  # N x 1 x V x C 형태의 배열 초기화 -> 출발 그룹 하나만 고려
for student_idx, s_row in source_group_df.iterrows(): # N으로 돌리기엔 s_row 정보가 없어서 iterrows 사용
    student_arrayidx = studentidx_to_arrayidx[student_idx] # df 인덱스와 행렬 인덱스가 다르기 때문에 사용
    for targetgroup_arrayidx in range(1): # 출발 그룹 하나만 고려
        for var in ['성별', '상담필요']:
            v_idx = var_to_idx[var]
            cat_value = s_row[var]
            c_idx = category_to_idx[var][cat_value]
            # 출발 그룹에서 해당 학생의 변수와 카테고리의 diff 값의 변화를 행렬에 반영
            after_source_diff_delta_matrix[student_arrayidx, targetgroup_arrayidx, v_idx, c_idx] = 1.0

In [145]:
print(after_source_diff_delta_matrix.shape)

(40, 1, 2, 2)


In [146]:
after_source_diff_delta_matrix # 각각의 학생이 출발 그룹에서 도착그룹으로 이동했을 때 출발 그룹의 diff 변화량

array([[[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[0., 1.],
         [1., 0.]]],


       [[[1., 0.],
         [1., 0.]]],


       [[[0., 1.],
         [0., 1.]]],


       [[[1., 0.],
         [0., 1

In [149]:
# 현재 모든 그룹의 diff_matrix 배열
# 이걸로 출발 그룹의 diff 변화량을 합산해 출발 그룹의 최종 diff_matrix 계산해야함
# 또한 도착 그룹의 diff 변화량 계산에도 사용해야함

diff_matrix.shape # (전체 그룹 수, 변수 수, 최대 카테고리 수)
# 출발 학생 수 고려해서 차원을 확정해야함

# 현재 모든 그룹의 diff_matrix 배열을 출발 학생 수에 맞게 확장
before_diff_matrix = diff_matrix[None, :, :] * np.ones((N, 1, 1, 1))  # N x G x V x C 형태로 확장
print(before_diff_matrix.shape)
# 근데 도착그룹 수만큼 차원을 줄여야함 (위와 같이 하면 출발그룹도 포함되어서 복잡해짐)
before_target_diff_matrix = before_diff_matrix[:, list(targetgroup_to_arrayidx.values()), :, :] # 도착 그룹의 diff_matrix만 남기기
# 확인용
print(before_target_diff_matrix.shape)

# 출발 그룹의 diff_matrix 계산 -> 출발 그룹 1개만 남기기
before_source_diff_matrix = before_diff_matrix[:, 1:2, :, :] # 출발 그룹의 diff_matrix만 남기기 -> 1로 넣으니 차원이 생략되서 1:2로 슬라이싱
print(before_source_diff_matrix.shape)

(40, 11, 2, 2)
(40, 10, 2, 2)
(40, 1, 2, 2)


In [151]:
# 이동 후 출발 그룹 diff_matrix 계산
move_source_diff_matrix = before_source_diff_matrix + after_source_diff_delta_matrix
print(move_source_diff_matrix.shape)
print(move_source_diff_matrix)

(40, 1, 2, 2)
[[[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [-0.54545455 -4.81818182]]]


 [[[-0.63636364 -4.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[-0.63636364 -4.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[-0.63636364 -4.72727273]
   [-0.54545455 -4.81818182]]]


 [[[-0.63636364 -4.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [-0.54545455 -4.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[-0.63636364 -4.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182]]]


 [[[ 0.36363636 -5.72727273]
   [ 0.45454545 -5.81818182

In [155]:
# 이동 후 도착 그룹의 diff_matrix 계산
move_target_diff_matrix = before_target_diff_matrix + after_target_diff_delta_matrix
print(move_target_diff_matrix.shape)
print(move_target_diff_matrix)

(40, 10, 2, 2)
[[[[  8.36363636   1.27272727]
   [  3.45454545   6.18181818]]

  [[ -1.63636364  -5.72727273]
   [ -1.54545455  -5.81818182]]

  [[  1.36363636   7.27272727]
   [  3.45454545   5.18181818]]

  ...

  [[  1.36363636  -7.72727273]
   [ -2.54545455  -3.81818182]]

  [[  0.36363636  -0.72727273]
   [  4.45454545  -4.81818182]]

  [[-12.63636364  -1.72727273]
   [-12.54545455  -1.81818182]]]


 [[[  8.36363636   1.27272727]
   [  3.45454545   6.18181818]]

  [[ -1.63636364  -5.72727273]
   [ -1.54545455  -5.81818182]]

  [[  1.36363636   7.27272727]
   [  3.45454545   5.18181818]]

  ...

  [[  1.36363636  -7.72727273]
   [ -2.54545455  -3.81818182]]

  [[  0.36363636  -0.72727273]
   [  4.45454545  -4.81818182]]

  [[-12.63636364  -1.72727273]
   [-12.54545455  -1.81818182]]]


 [[[  8.36363636   1.27272727]
   [  4.45454545   5.18181818]]

  [[ -1.63636364  -5.72727273]
   [ -0.54545455  -6.81818182]]

  [[  1.36363636   7.27272727]
   [  4.45454545   4.18181818]]

  ...



In [156]:
# move_source_diff_matrix와 move_target_diff_matrix를 이용해
# 이동 후 최종 diff_matrix 계산
'''
move_source_diff_matrix, move_target_diff_matrix 차원이 달라서 문제가 발생함
move_source_diff_matrix: N x 1 x V x C -> 출발그룹만 존재
move_target_diff_matrix: N x G x V x C -> 도착그룹들
move_target_diff_matrix에서 각 학생이 도착그룹으로 이동 시 각 그룹별로 총 diff 값을 계산해서 제일 작읍 그룹하나만 산출해
차원을 줄이면서 move_source_diff_matrix와 차원을 통일시키고 싶은데, 이렇게 되면 도착그룹 배열 인덱스 정보가 사라지면서 문제가 발생함

- 해결책
1. 학생 마다 도착 그룹별 총 비용을 계산해서
2. 학생 마다 최적 도착그룹 index를 별도 배열에 저장
'''

'\nmove_source_diff_matrix, move_target_diff_matrix 차원이 달라서 문제가 발생함\nmove_source_diff_matrix: N x 1 x V x C -> 출발그룹만 존재\nmove_target_diff_matrix: N x G x V x C -> 도착그룹들\nmove_target_diff_matrix에서 각 학생이 도착그룹으로 이동 시 각 그룹별로 총 diff 값을 계산해서 제일 작읍 그룹하나만 산출해\n차원을 줄이면서 move_source_diff_matrix와 차원을 통일시키고 싶은데, 이렇게 되면 도착그룹 배열 인덱스 정보가 사라지면서 문제가 발생함\n\n- 해결책\n1. 학생 마다 도착 그룹별 총 비용을 계산해서\n2. 학생 마다 최적 도착그룹 index를 별도 배열에 저장\n'

In [160]:
# 학생별 도착 그룹 비용을 저장하는 별도 배열 생성
move_target_totaldiff_matrix = np.zeros((N, G), dtype=float)  # N x G 형태의 배열 초기화
move_target_totaldiff_matrix

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 

In [166]:
print(move_target_diff_matrix[0, :, :, :]) # 첫번째 학생이 각 도착 그룹으로 이동했을 때의 diff_matrix
print(("\n"))

[[[  8.36363636   1.27272727]
  [  3.45454545   6.18181818]]

 [[ -1.63636364  -5.72727273]
  [ -1.54545455  -5.81818182]]

 [[  1.36363636   7.27272727]
  [  3.45454545   5.18181818]]

 [[ -5.63636364   3.27272727]
  [ -1.54545455  -0.81818182]]

 [[  2.36363636   3.27272727]
  [  1.45454545   4.18181818]]

 [[ -1.63636364   1.27272727]
  [  2.45454545  -2.81818182]]

 [[ -2.63636364  -0.72727273]
  [ -6.54545455   3.18181818]]

 [[  1.36363636  -7.72727273]
  [ -2.54545455  -3.81818182]]

 [[  0.36363636  -0.72727273]
  [  4.45454545  -4.81818182]]

 [[-12.63636364  -1.72727273]
  [-12.54545455  -1.81818182]]]




In [167]:
print(move_target_diff_matrix[0, 0, :, :]) # 첫번째 학생이 첫번째 도착 그룹으로 이동했을 때의 diff_matrix
print(("\n"))

[[8.36363636 1.27272727]
 [3.45454545 6.18181818]]




In [170]:
# 각 원소의 제곱하여 총합
test = move_target_diff_matrix ** 2
print(test[0, 0, :, :])
print(np.sum(test[0, 0, :, :]))

[[69.95041322  1.61983471]
 [11.9338843  38.21487603]]
121.7190082644628
